# **LoanGuard: Data Cleaning**

### **Overall Goal**:

- The Goal is to clean the data by:
    - Performing Data Checks
        - Data types
        - Duplicate Data
        - Missing values
        - Outliers
    - Handling Categorical Columns
    - Split and Save data

### 1. **Importing required packages**

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import hvplot.pandas

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

### 2. **Loading the data and having a first look at it!**

#### Load the data

In [2]:
df = pd.read_csv("../data/raw/data.csv")

#### Shape of the dataset

In [3]:
df.shape

(396030, 27)

#### First 5 rows

In [4]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.00,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.00,Not Verified,Jan-2015,Fully Paid,vacation,Vacation,26.24,Jun-1990,16.00,0.00,36369.00,41.80,25.00,w,INDIVIDUAL,0.00,0.00,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.00,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.00,Not Verified,Jan-2015,Fully Paid,debt_consolidation,Debt consolidation,22.05,Jul-2004,17.00,0.00,20131.00,53.30,27.00,f,INDIVIDUAL,3.00,0.00,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.00,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.00,Source Verified,Jan-2015,Fully Paid,credit_card,Credit card refinancing,12.79,Aug-2007,13.00,0.00,11987.00,92.20,26.00,f,INDIVIDUAL,0.00,0.00,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.00,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.00,Not Verified,Nov-2014,Fully Paid,credit_card,Credit card refinancing,2.60,Sep-2006,6.00,0.00,5472.00,21.50,13.00,f,INDIVIDUAL,0.00,0.00,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.00,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.00,Verified,Apr-2013,Charged Off,credit_card,Credit Card Refinance,33.95,Mar-1999,13.00,0.00,24584.00,69.80,43.00,f,INDIVIDUAL,1.00,0.00,"679 Luna Roads\r\nGreggshire, VA 11650"


### 3. **Performing Data Checks**

#### 3.1 **Check and Fix Data Types**

In [5]:
# checking null and Dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394274 non-null  object 
 15  

##### Fixing the dtypes

In [6]:
df['term'] = df['term'].astype(str).str.extract(r'(\d+)').astype(float)
df['int_rate'] = df['int_rate'].astype(str).str.replace('%', '').astype(float)
df['revol_util'] = df['revol_util'].astype(str).str.replace('%', '').astype(float)
df['emp_length'] = df['emp_length'].replace({'10+ years': 10, '< 1 year': 0}).astype(str).str.extract(r'(\d+)').astype(float)

#### 3.2 **Checking duplicate data**

In [7]:
df.duplicated().sum()

0

There are no duplicates data either

#### 3.3 **Handle Missing Values**

In [8]:
for column in df.columns:
    if df[column].isna().sum() != 0:
        missing = df[column].isna().sum()
        portion = (missing / df.shape[0]) * 100
        print(f"'{column}': number of missing values '{missing}' ==> '{portion:.3f}%'")

'emp_title': number of missing values '22927' ==> '5.789%'
'emp_length': number of missing values '18301' ==> '4.621%'
'title': number of missing values '1756' ==> '0.443%'
'revol_util': number of missing values '276' ==> '0.070%'
'mort_acc': number of missing values '37795' ==> '9.543%'
'pub_rec_bankruptcies': number of missing values '535' ==> '0.135%'


emp_title

In [9]:
df.emp_title.nunique()

173105

Realistically there are too many unique job titles to try to convert this to a dummy variable feature. Let's remove that emp_title column.

In [10]:
df.drop('emp_title', axis=1, inplace=True)

emp_length

In [11]:
df.emp_length.unique()

array([10.,  4.,  0.,  6.,  9.,  2.,  3.,  8.,  7.,  5.,  1., nan])

In [12]:
for year in df.emp_length.unique():
    print(f"{year} years in this position:")
    print(f"{df[df.emp_length == year].loan_status.value_counts(normalize=True)}")
    print('==========================================')

10.0 years in this position:
loan_status
Fully Paid    0.82
Charged Off   0.18
Name: proportion, dtype: float64
4.0 years in this position:
loan_status
Fully Paid    0.81
Charged Off   0.19
Name: proportion, dtype: float64
0.0 years in this position:
loan_status
Fully Paid    0.79
Charged Off   0.21
Name: proportion, dtype: float64
6.0 years in this position:
loan_status
Fully Paid    0.81
Charged Off   0.19
Name: proportion, dtype: float64
9.0 years in this position:
loan_status
Fully Paid    0.80
Charged Off   0.20
Name: proportion, dtype: float64
2.0 years in this position:
loan_status
Fully Paid    0.81
Charged Off   0.19
Name: proportion, dtype: float64
3.0 years in this position:
loan_status
Fully Paid    0.80
Charged Off   0.20
Name: proportion, dtype: float64
8.0 years in this position:
loan_status
Fully Paid    0.80
Charged Off   0.20
Name: proportion, dtype: float64
7.0 years in this position:
loan_status
Fully Paid    0.81
Charged Off   0.19
Name: proportion, dtype: float64


Charge off rates are extremely similar across all employment lengths. So we are going to drop the emp_length column.

In [13]:
df.drop('emp_length', axis=1, inplace=True)

title

In [14]:
df.title.value_counts().head()

title
Debt consolidation         152472
Credit card refinancing     51487
Home improvement            15264
Other                       12930
Debt Consolidation          11608
Name: count, dtype: int64

In [15]:
df.purpose.value_counts().head()

purpose
debt_consolidation    234507
credit_card            83019
home_improvement       24030
other                  21185
major_purchase          8790
Name: count, dtype: int64

The title column is simply a string subcategory/description of the purpose column. So we are going to drop the title column.

In [16]:
df.drop('title', axis=1, inplace=True)

**mort_acc**
- There are many ways we could deal with this missing data. We could attempt to build a simple model to fill it in, such as a linear model, we could just fill it in based on the mean of the other columns, or you could even bin the columns into categories and then set NaN as its own category. There is no 100% correct approach!

- Let's review the other columns to see which most highly correlates to mort_acc

In [17]:
df.mort_acc.value_counts()

mort_acc
0.00     139777
1.00      60416
2.00      49948
3.00      38049
4.00      27887
5.00      18194
6.00      11069
7.00       6052
8.00       3121
9.00       1656
10.00       865
11.00       479
12.00       264
13.00       146
14.00       107
15.00        61
16.00        37
17.00        22
18.00        18
19.00        15
20.00        13
24.00        10
22.00         7
21.00         4
25.00         4
27.00         3
32.00         2
31.00         2
23.00         2
26.00         2
28.00         1
30.00         1
34.00         1
Name: count, dtype: int64

In [18]:
df.mort_acc.isna().sum()

37795

In [19]:
numerical_df = df.select_dtypes(include=['float64', 'int64'])
numerical_df.corr()['mort_acc'].drop('mort_acc').sort_values().hvplot.barh(title="Correlation of 'mort_acc' with other variables", xlabel="Correlation", ylabel="Feature")

:Bars   [index]   (mort_acc)

Looks like the total_acc feature correlates with the mort_acc , this makes sense! Let's try this fillna() approach. We will group the dataframe by the total_acc and calculate the mean value for the mort_acc per total_acc entry. To get the result below:

In [20]:
total_acc_avg = df.groupby(by='total_acc')['mort_acc'].mean()

In [21]:
def fill_mort_acc(total_acc, mort_acc):
    if np.isnan(mort_acc):
        return total_acc_avg[total_acc].round()
    else:
        return mort_acc

In [22]:
df['mort_acc'] = df.apply(lambda x: fill_mort_acc(x['total_acc'], x['mort_acc']), axis=1)

**revol_util & pub_rec_bankruptcies**

- These two features have missing data points, but they account for less than 0.5% of the total data. So we are going to remove the rows that are missing those values in those columns with dropna().

In [23]:
for column in df.columns:
    if df[column].isna().sum() != 0:
        missing = df[column].isna().sum()
        portion = (missing / df.shape[0]) * 100
        print(f"'{column}': number of missing values '{missing}' ==> '{portion:.3f}%'")

'revol_util': number of missing values '276' ==> '0.070%'
'pub_rec_bankruptcies': number of missing values '535' ==> '0.135%'


In [24]:
df.dropna(inplace=True)

In [25]:
df.shape

(395219, 24)

#### 3.4 **Handle Outliers**

In [26]:
df.describe()

,loan_amnt,term,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00,395219.00
mean,14122.06,41.71,13.64,432.07,74199.37,17.39,11.32,0.18,15851.65,53.81,25.43,1.79,0.12
std,8357.05,10.22,4.47,250.70,61557.25,18.03,5.13,0.53,20584.27,24.44,11.89,2.07,0.36
min,500.00,36.00,5.32,16.08,0.00,0.00,1.00,0.00,0.00,0.00,2.00,0.00,0.00
25%,8000.00,36.00,10.49,250.33,45000.00,11.30,8.00,0.00,6038.00,35.90,17.00,0.00,0.00
50%,12000.00,36.00,13.33,375.49,64000.00,16.92,10.00,0.00,11190.00,54.80,24.00,1.00,0.00
75%,20000.00,36.00,16.55,567.79,90000.00,22.99,14.00,0.00,19626.00,72.90,32.00,3.00,0.00
max,40000.00,60.00,30.99,1533.81,8706582.00,9999.00,90.00,86.00,1743266.00,892.30,151.00,34.00,8.00


We'll cap extreme values beyond the 95th percentile.

- note:
    - `loan_amnt` has mean of $14k and max is $40k which is extreme.
    - `annual_inc` has a huge max of $8.7M which is extreme.
    - `revol_bal` has a max of $1.7M which is far away from 75%.

- Why not Other cols?
    1. Columns like `dti`, `pub_rec`, `mort_acc`, etc., don’t have such extreme variances.
    2. `pub_rec_bankruptcies` max is 8, which is rare but not unmanageable.
    3. `total_acc` and `open_acc` are within reasonable bounds (max 151 and 90, respectively).
    4. `int_rate` (interest rate) max is 30.99%, which is high but still within real-world lending terms.

In [27]:
df[['loan_amnt', 'annual_inc', 'revol_bal']].hvplot.box(title="Outlier Visualization (Before)")

:BoxWhisker   [Variable]   (value)

In [28]:
for col in ['loan_amnt', 'annual_inc', 'revol_bal']:
    upper_limit = df[col].quantile(0.95)
    df[col] = np.where(df[col] > upper_limit, upper_limit, df[col])

In [29]:
df[['loan_amnt', 'annual_inc', 'revol_bal']].hvplot.box(title="Outlier Visualization (After)")

:BoxWhisker   [Variable]   (value)

### 4. **Handling Categorical Columns**

In [30]:
print([column for column in df.columns if df[column].dtype == object])

['grade', 'sub_grade', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'earliest_cr_line', 'initial_list_status', 'application_type', 'address']


#### 4.1 **grade & sub_grade**
- We know that grade is just a sub feature of sub_grade, So we are goinig to drop it.

In [31]:
df.drop('grade', axis=1, inplace=True)

In [32]:
dummies = ['sub_grade', 'verification_status', 'purpose', 'initial_list_status', 'application_type', 'home_ownership']

df = pd.get_dummies(df, columns=dummies, drop_first=True)

#### 4.2 **address**
- We are going to feature engineer a zip code column from the address in the data set. Create a column called 'zip_code' that extracts the zip code from the address column.

In [33]:
df.address.head()

0       0174 Michelle Gateway\r\nMendozaberg, OK 22690
1    1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113
2    87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113
3              823 Reid Ford\r\nDelacruzside, MA 00813
4               679 Luna Roads\r\nGreggshire, VA 11650
Name: address, dtype: object

In [34]:
df['zip_code'] = df.address.apply(lambda x: x[-5:])

In [35]:
df.zip_code.value_counts()

zip_code
70466    56880
22690    56413
30723    56402
48052    55811
00813    45725
29597    45393
05113    45300
11650    11210
93700    11126
86630    10959
Name: count, dtype: int64

In [36]:
df = pd.get_dummies(df, columns=['zip_code'], drop_first=True)

In [37]:
df.drop('address', axis=1, inplace=True)

#### 4.3 **issue_d**
- This would be data leakage, we wouldn't know beforehand whether or not a loan would be issued when using our model, so in theory we wouldn't have an issue_date, drop this feature.

In [38]:
df.drop('issue_d', axis=1, inplace=True)

#### 4.4 **earliest_cr_line**
- This appears to be a historical time stamp feature. Extract the year from this feature using a .apply() function, then convert it to a numeric feature.

In [39]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y', errors='coerce')
df['earliest_cr_line'] = df.earliest_cr_line.dt.year

In [40]:
df.earliest_cr_line.nunique()

65

In [41]:
df.earliest_cr_line.value_counts()

earliest_cr_line
2000    29302
2001    29031
1999    26444
2002    25849
2003    23623
        ...  
1951        3
1950        3
1953        2
1944        1
1948        1
Name: count, Length: 65, dtype: int64

### 5. **Split and Save the cleaned data**

In [42]:
train, test = train_test_split(df, test_size=0.33, random_state=42)

In [43]:
print(f"Training set shape: {train.shape}")
print(f"Testing set shape: {test.shape}")

Training set shape: (264796, 81)
Testing set shape: (130423, 81)


In [44]:
print(train.shape)
train = train[train['annual_inc'] <= 250000]
train = train[train['dti'] <= 50]
train = train[train['open_acc'] <= 40]
train = train[train['total_acc'] <= 80]
train = train[train['revol_util'] <= 120]
train = train[train['revol_bal'] <= 250000]
print(train.shape)

(264796, 81)
(264463, 81)


In [45]:
# define numerical & categorical columns
numerical_features = [feature for feature in df.columns if df[feature].dtype != 'O']
categorical_features = [feature for feature in df.columns if df[feature].dtype == 'O']

# print columns
print('We have {} numerical features : {}'.format(len(numerical_features), numerical_features))
print('\nWe have {} categorical features : {}'.format(len(categorical_features), categorical_features))

We have 80 numerical features : ['loan_amnt', 'term', 'int_rate', 'installment', 'annual_inc', 'dti', 'earliest_cr_line', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'mort_acc', 'pub_rec_bankruptcies', 'sub_grade_A2', 'sub_grade_A3', 'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2', 'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1', 'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5', 'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4', 'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3', 'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2', 'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1', 'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5', 'verification_status_Source Verified', 'verification_status_Verified', 'purpose_credit_card', 'purpose_debt_consolidation', 'purpose_educational', 'purpose_home_improvement', 'purpose_house', 'purpose_major_purchase', 'purpose_medical'

#### 6.1 **Saving the data**

In [46]:
train.to_csv('../data/processed/train.csv', index=False)
test.to_csv('../data/processed/test.csv', index=False)